# Astar dataset

The puprose of this dataset is to explore learning end to end the Astar algorithm from scratch.
This dataset provides the enviroment defined as a grid, where:
- 0 indicates free space, and 1 indicates obstacle.
- Not all enviroments have a solution.
- The solution is defined as a sequence of actions, where:
   0 -> up, 1 -> right, 2 -> down, 3 -> left
- The solution is given as a series of actions [0, 1, 0, 2]

## The input

The input is of the shape 3 x n x m, where:
- n: height of the grid for planning
- m: width of the grid for planning
- The 3 nxm grids are: 1. The grid representing the environment, 2. The one hot encoded vector representing the start, 3. The one hot encoded vector representing the goal

## The output
The output will be 5 classes:
- Move up (0)
- Move down (1)
- Move left (2)
- Move right (3)
- Do nothing (4) (no solution exists)
The problem can be treated as a classification problem, where each of the above actions can have a specific probability.


There are 2 problems that this dataset can help solve:
1. Whether a solution exits or not (completeness)
2. Finiding the optimal solution if it exists

In [ ]:
# Set the grid size here
n, m = 10, 10  # grid size for the problem
N = 10000      # Number of examples

# Probability of existence of obstacle
obstacle_probability = 0.2

In [ ]:
from datasets.astar_dataset import make_astar_dataset
import time
# Create the data set
start = time.time()
X, y = make_astar_dataset(N, n, m, obstacle_probability)
print(f"Execution time: {time.time() - start:.4f} seconds")

In [ ]:
from astar_dataset import plot_instance
print(y[1])
plot_instance(X[1])

# TODO:
1. Make the label distribution uniform, use data augmentation to do so (rotate the grid .. etc)
2. Add more examples where there is no path that could be found